In [1]:
import pandas as pd
import numpy as np
import os
from encoding_function import low_frequency_to_others
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings('ignore')

### 데이터 불러오기
- 학습: 2023년도 이전
- 검증: 2023년
- 테스트: 2024년

In [2]:
def add_y(df):
    df['rank'] = df['rank'].replace(0, 6)

    target_cols = [
        '단승', '복승', '삼복승'
    ]
    for i, col in enumerate(target_cols):
        condition_target = df['rank'] <= i+1
        df.loc[condition_target, col] = 1

    df[target_cols] = df[target_cols].fillna(0)
    df.drop(['rank'], axis=1, inplace=True)
    return df


ROOT_DIR = "data"
RANDOM_STATE = 42

train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

train['성별'] = train['성별'].map({'남': 0, '여': 1})
test['성별'] = test['성별'].map({'남': 0, '여': 1})

train = add_y(train)
test = add_y(test)

print(train.shape, test.shape)
train.head()

(79950, 45) (6102, 45)


,Race_ID,연도,회차,일차,경주번호,번호,등급,기수,선수명,성별,...,모터_연대율3,전탑승선수1,전탑승선수2,보트번호,보트_평균착순점,보트_연대율,특이사항,단승,복승,삼복승
0,2013_1_1_1,2013,1,1,1,1,A2,2,이용세,0,...,46.3,김신오/25,김선필/25,10,5.60,32.8,NaN,0.0,1.0,1.0
1,2013_1_1_1,2013,1,1,1,2,B1,7,손동민,0,...,70.9,김승택/3,경상수/1316,68,5.59,40.7,NaN,0.0,0.0,1.0
2,2013_1_1_1,2013,1,1,1,3,B2,11,박진서,0,...,55.7,송종해/45,문주엽/4,22,5.07,30.7,NaN,0.0,0.0,0.0
3,2013_1_1_1,2013,1,1,1,4,B1,4,경상수,0,...,75.0,이종한/4,문주엽/223,77,5.49,39.7,NaN,1.0,1.0,1.0
4,2013_1_1_1,2013,1,1,1,5,B1,8,정주현,0,...,22.4,이창규/6,반혜진/545,18,5.69,37.6,NaN,0.0,0.0,0.0


In [4]:
def add_weight_penalty(df):
    weight_limit_male = 55
    weight_limit_female = 51
    df['체중'] = df['체중'].astype(float)
    
    # 중량 부과 여부 계산 (성별에 따라 다른 기준 적용)
    df['중량부과여부'] = df.apply(lambda row: 1 if (row['성별'] == 0 and row['체중'] < weight_limit_male) or
                                             (row['성별'] == 1 and row['체중'] < weight_limit_female) else 0, axis=1)

    # 부과된 중량 계산 (성별에 따라 차감된 체중을 계산)
    df['부과된중량'] = df.apply(lambda row: max(0, weight_limit_male - row['체중']) if row['성별'] == 0 else
    max(0, weight_limit_female - row['체중']), axis=1)
    
    df.drop('체중', axis=1, inplace=True)

    return df

train = add_weight_penalty(train)
test = add_weight_penalty(test)

In [5]:
def bin_age(df):
    # 나이를 5개 구간으로 나누어 bin화 (예: 0-20, 20-30, 30-40, 40-50, 50+)
    bins = [20, 30, 40, 50, 100]
    labels = ['20-30', '30-40', '40-50', '50+']

    # 나이 변수를 bin화
    df['나이'] = pd.cut(df['나이'], bins=bins, labels=labels, right=False)

    return df

train = bin_age(train)
test = bin_age(test)

## 사용하지 않는 feature drop

- 전일 성적을 어떻게 반영할지

In [6]:
def drop_columns_from_datasets(df):
    drop_cols = [
        '연도', '회차', '일차', '경주번호',
        '금일출주경주',
        '모터번호', '전탑승선수1', '전탑승선수2',
        '보트번호', '특이사항',
        'FL' # F뒤의 숫자는 현재 반기의 실격 횟수, L뒤의 숫자는 반칙횟수?
    ]

    df = df.drop(drop_cols, axis=1)

    return df


train = drop_columns_from_datasets(train)
test = drop_columns_from_datasets(test)

print(train.shape, test.shape)

(79950, 36) (6102, 35)


In [7]:
def reverse_rank_values(df_train, df_val):
    cols_to_reverse = [
        '최근6회차_평균착순점', '최근6회차_평균득점',
        '연간성적_평균착순점',
        '모터_평균착순점',
        '보트_평균착순점'
    ]
    for col in cols_to_reverse:
        max_rank = df_train[col].max()
        # print(f'{col}: {max_rank}')
        df_train[col] = max_rank - df_train[col]
        df_val[col] = max_rank - df_val[col]
    
        df_train[col].fillna(0, inplace=True)
        df_val[col].fillna(0, inplace=True)
    
    return df_train, df_val


train, test = reverse_rank_values(train, test)

In [8]:
import re

def extract_numbers(result):
    if isinstance(result, str):  # result가 문자열인 경우에만 처리
        return re.findall(r'-(\d+)-', result)  # -숫자- 형식에서 중간 숫자 추출
    return []  # 문자열이 아니면 빈 리스트 반환

def calculate_mean(numbers):
    numbers = [int(num) for num in numbers]  # 리스트 안의 문자열 숫자를 정수로 변환
    if len(numbers) > 0:
        return np.mean(numbers)  # 리스트가 비어있지 않으면 평균 계산
    else:
        return np.nan  # 빈 리스트인 경우 NaN 반환

def last_race_process(df_train, df_val):
    df_train['전일성적'] = df_train['전일성적'].apply(extract_numbers)  # 순위만 추출 (코스도 같이 추출?)
    df_train['전일성적'] = df_train['전일성적'].apply(calculate_mean) # 평균 계산

    df_val['전일성적'] = df_val['전일성적'].apply(extract_numbers)
    df_val['전일성적'] = df_val['전일성적'].apply(calculate_mean)

    max_rank = df_train['전일성적'].max()
    df_train['전일성적'] = max_rank - df_train['전일성적'] + 1
    df_val['전일성적'] = max_rank - df_val['전일성적'] + 1
    
    df_train['전일성적'].fillna(1, inplace=True)
    df_val['전일성적'].fillna(1, inplace=True)
    
    return df_train, df_val

train, test = last_race_process(train, test)

## 일부 숫자형 변수 변환

- 코스별 성적/경기수 분리

In [9]:
def separation_course(df):
    col_list = [
        '코스_1코스', '코스_2코스', '코스_3코스', '코스_4코스', '코스_5코스', '코스_6코스'
    ]
    for col in col_list:
        df[[f'{col[3:]}_성적', f'{col[3:]}_경기수']] = df[col].fillna('').str.split('/', expand=True)

    df.drop(col_list, axis=1, inplace=True)

    return df
        


train = separation_course(train)
test = separation_course(test)

print(train.shape, test.shape)

(79950, 42) (6102, 41)


- 코스별 성적 스무딩

In [10]:
def apply_laplace_smoothing(df, col, global_mean, alpha):
    # 경기수 0인 값이 너무 높게 나오는 경향이 있어 분모에 상수 1 추가(없애도 됨)
    encoded_value = (df[f'{col}_성적'] * df[f'{col}_경기수'] + global_mean * alpha) / (1 + df[f'{col}_경기수'] + alpha)
    df[f'{col}_성적'] = encoded_value

    return df

def laplace_smoothing_to_course(train, val=None, alpha=1):
    col_list = [
        '1코스', '2코스', '3코스', '4코스', '5코스', '6코스'
    ]
    for col in col_list:
        train[f'{col}_성적'] = train[f'{col}_성적'].astype(float)
        train[f'{col}_경기수'] = train[f'{col}_경기수'].astype(float)
        if val is not None:
            val[f'{col}_성적'] = val[f'{col}_성적'].astype(float)
            val[f'{col}_경기수'] = val[f'{col}_경기수'].astype(float)

    # Train 데이터에서 글로벌 평균 계산
    global_means = {col: train[f'{col}_성적'].mean() for col in col_list}

    for col in col_list:
        # Train 데이터에 라플라스 스무딩 적용
        train = apply_laplace_smoothing(train, col, global_means[col], alpha)
        train.drop(f'{col}_경기수', axis=1, inplace=True)

    if val is not None:
        for col in col_list:
            # Validation 데이터에 Train에서 구한 글로벌 평균으로 라플라스 스무딩 적용
            val = apply_laplace_smoothing(val, col, global_means[col], alpha)
            val.drop(f'{col}_경기수', axis=1, inplace=True)

    if val is not None:
        return train, val
    else:
        return train


train, test = laplace_smoothing_to_course(train, test, alpha=1) # 알파가 작을수록 빈도수에 가깝세, 알파가 클수록 전체 평균에 가깝게
print(train.shape, test.shape)

(79950, 36) (6102, 35)


In [11]:
def set_course_scores(df):
    # '코스_성적' 열을 초기화
    df['코스_성적'] = 0

    # '번호' 열에 따른 '코스_성적' 값 설정
    for i in range(1, 7):
        mask = df['번호'] == i
        df.loc[mask, '코스_성적'] = df.loc[mask, f'{i}코스_성적']

    # 제거할 열 목록
    drop_cols = [f'{i}코스_성적' for i in range(1, 7)]

    # 열 제거
    df = df.drop(columns=drop_cols)

    return df


from sklearn.decomposition import PCA
def apply_pca(df_train, df_val, n_components=2):
    course_cols = [f'{i}코스_성적' for i in range(1, 7)]

    pca = PCA(n_components=n_components)
    pca.fit(df_train[course_cols])

    # 훈련 데이터에 PCA 변환 적용 (transform)
    train_pca = pca.transform(df_train[course_cols])
    val_pca = pca.transform(df_val[course_cols])
    for i in range(n_components):
        col_name = f'PCA_코스성적_{i+1}'
        df_train[col_name] = train_pca[:, i]
        df_val[col_name] = val_pca[:, i]

    # 각 컴포넌트별 분산 설명 비율 출력
    # explained_variance_ratios = pca.explained_variance_ratio_
    # for i, ratio in enumerate(explained_variance_ratios):
    #     print(f"PCA Component {i+1}: {ratio:.4f} variance explained")
    
    # n_components=2가 제일 좋아보임

    return df_train, df_val


train, test = apply_pca(train, test, n_components=2)
train = set_course_scores(train)
test = set_course_scores(test)

- 최근 8경기 착순 분리

In [12]:
def split_last_eight_rank(df):
    for i in range(0, 4):
        df[f'최근{i+1}경기_착순'] = df['최근8경주_착순'].str[i]
    for j in range(5, 9):
        df[f'최근{j}경기_착순'] = df['최근8경주_착순'].str[j]
        
    df.drop('최근8경주_착순', axis=1, inplace=True)
    
    return df

# def adjust_for_top3(df):
#     col_list = [
#         '최근1경기_착순', '최근2경기_착순', '최근3경기_착순', '최근4경기_착순',
#         '최근5경기_착순', '최근6경기_착순', '최근7경기_착순', '최근8경기_착순'
#     ]
# 
#     for col in col_list:
#     # 순위가 1, 2, 3이 아닌 경우, 결측인 경우, 6으로 조정
#     # (일반화된 성능을 위해 + 3등내에 드는게 중요)
#         df[col] = df[col].apply(lambda x: x if x in ['1', '2', '3'] else '-1')
# 
#     return df

def adjust_last_eight_rank(df_train, df_val):
    col_list = [
        '최근1경기_착순', '최근2경기_착순', '최근3경기_착순', '최근4경기_착순',
        '최근5경기_착순', '최근6경기_착순', '최근7경기_착순', '최근8경기_착순'
    ]

    for col in col_list:
        df_train[col] = df_train[col].fillna(6).astype(int)
        df_val[col] = df_val[col].fillna(6).astype(int)
        
        df_train[col] = df_train[col].replace(0, 6)  # 가끔씩 0이 있는 경우가 존재
        df_val[col] = df_val[col].replace(0, 6)

        max_rank = df_train[col].max()
        df_train[col] = max_rank - df_train[col]
        df_val[col] = max_rank - df_val[col]
    
    first_mean_cols = 3
    col_to_mean_1 = col_list[:first_mean_cols]

    df_train[f'최근{first_mean_cols}경기_평균'] = df_train[col_to_mean_1].mean(axis=1)
    df_val[f'최근{first_mean_cols}경기_평균'] = df_val[col_to_mean_1].mean(axis=1)

    df_train.drop(col_list, axis=1, inplace=True)
    df_val.drop(col_list, axis=1, inplace=True)

    return df_train, df_val


train = split_last_eight_rank(train)
# train = adjust_for_top3(train)

test = split_last_eight_rank(test)
# test = adjust_for_top3(test)

train, test = adjust_last_eight_rank(train, test)

print(train.shape, test.shape)

(79950, 33) (6102, 32)


## 범주형 변수 확인


In [13]:
def cal_cat_cols(train, val=None):
    objective_cols = []

    # 특정 문자열이 포함된 열을 범주형 변수로 지정
    # cat_kerword_list = ['번호', '기수', '경기_착순']
    cat_kerword_list = ['번호']
    for col in train.columns:
        if any(sub in col for sub in cat_kerword_list):
            objective_cols.append(col)
            train[col] = train[col].astype('str')
            val[col] = val[col].astype('str')
            
    # 나머지 열에 대해 숫자형 변환 시도
    for col in train.columns:
        if col in objective_cols:
            continue  # 이미 범주형으로 처리된 열은 제외
        try:
            # 'float' 타입으로 변환 시도
            train[col] = train[col].astype('float')
            val[col] = val[col].astype('float')
        except:
            objective_cols.append(col)

    cat_features = list(set(objective_cols) - set(['rank', 'Race_ID']))
    cat_features = [feature for feature in cat_features if '단승' not in feature]
    cat_features = [feature for feature in cat_features if '복승' not in feature]
    cat_features = [feature for feature in cat_features if '삼복승' not in feature]

    return cat_features


cat_features = cal_cat_cols(train, test)
cat_features

['등급', '번호', '선수명', 'zero_included', '나이']

## X, y 분리

In [14]:
drop_cols = [
    'Race_ID', '번호', '단승', '복승', '삼복승', '선수명', '기수'
]

X_train = train.drop(drop_cols, axis=1)
y_train = train[['단승']]
X_test = test.drop(drop_cols, axis=1)
y_test = test[['단승']]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(79950, 26) (79950, 1) (6102, 25) (6102, 1)


In [15]:
def reshape_race_data(df, players_per_race=6):
    # numpy 배열로 변환
    values = df.values

    # 데이터 형태 변환 (6개의 행을 1개의 행으로)
    reshaped_values = values.reshape(-1, players_per_race * values.shape[1])

    # 새로운 컬럼 이름 생성
    columns = [f'{col}_{i+1}번선수' for i in range(players_per_race) for col in df.columns]

    # 재구성된 DataFrame 생성
    reshaped_df = pd.DataFrame(reshaped_values, columns=columns)

    return reshaped_df

X_train = reshape_race_data(X_train, players_per_race=6)
y_train = reshape_race_data(y_train, players_per_race=6)
y_train = np.array(y_train).astype(int)

X_test = reshape_race_data(X_test, players_per_race=6)
y_test = reshape_race_data(y_test, players_per_race=6)
y_test = np.array(y_test).astype(int)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(13325, 156) (13325, 6) (1017, 150) (1017, 6)


In [16]:
def low_to_others(train, val, threshold=5, is_train=True, verbose=False):
    cat_features = cal_cat_cols(train, val)
    if is_train==False:
        cat_features = list(set(cat_features) - set(['Race_ID_1번선수', 'Race_ID_2번선수', 'Race_ID_3번선수', 'Race_ID_4번선수', 'Race_ID_5번선수', 'Race_ID_6번선수']))

    for col in cat_features:
        unifier = low_frequency_to_others(threshold=threshold, verbose=verbose)
        train[col] = unifier.fit_transform(train[col])
        val[col] = unifier.transform(val[col])

    return train, val

X_train, X_test = low_to_others(X_train, X_test, threshold=5, verbose=False)

KeyError: 'zero_included_3번선수'

In [17]:
def all_precoess(train, val, target='단승', is_train=True):
    train['성별'] = train['성별'].map({'남': 0, '여': 1})
    val['성별'] = val['성별'].map({'남': 0, '여': 1})
    
    train = add_y(train)
    if is_train:
        val = add_y(val)

    train = add_weight_penalty(train) # 연승 정확도를 높이려면 제거
    val = add_weight_penalty(val) 

    train = bin_age(train)
    val = bin_age(val)
    
    train = drop_columns_from_datasets(train)
    val = drop_columns_from_datasets(val)

    train, val = last_race_process(train, val)

    train, val = reverse_rank_values(train, val)

    train = separation_course(train)
    val = separation_course(val)

    train, val = laplace_smoothing_to_course(train, val, alpha=1)

    train, val = apply_pca(train, val, n_components=2)
    
    train = set_course_scores(train)
    val = set_course_scores(val)
    
    train = split_last_eight_rank(train)
    # train = adjust_for_top3(train)
    
    val = split_last_eight_rank(val)
    # val = adjust_for_top3(val)

    train, val = adjust_last_eight_rank(train, val)

    if is_train:
        drop_cols = [
            'Race_ID', '번호', '단승', '복승', '삼복승', '선수명', '기수'
        ]
    else:
        drop_cols = [
            '번호', '선수명', '기수'
        ]
    
    X_train = train.drop(drop_cols, axis=1)
    y_train = train[[target]]
    X_train = reshape_race_data(X_train, players_per_race=6)
    y_train = reshape_race_data(y_train, players_per_race=6)
    y_train = np.array(y_train).astype(int)

    X_val = val.drop(drop_cols, axis=1)
    X_val = reshape_race_data(X_val, players_per_race=6)
    if is_train:
        y_val = val[[target]]
        y_val = reshape_race_data(y_val, players_per_race=6)
        y_val = np.array(y_val).astype(int)

    X_train, X_val = low_to_others(X_train, X_val, threshold=5, is_train=is_train, verbose=False)
    
    if is_train:
        return X_train, y_train, X_val, y_val
    else:
        return X_train, y_train, X_val
    
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

X_train, y_train, X_test, y_test = all_precoess(train, test, is_train=True)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(13325, 150) (13325, 6) (1017, 150) (1017, 6)


In [18]:
# ### 테스트용
# 
# from ctgan import CTGAN
# 
# # CTGAN 모델 생성
# ctgan = CTGAN(verbose=True)
# 
# cat_features = cal_cat_cols(X_train, X_test)
# 
# # 모델 학습 (target도 다른 변수와 함께 사용됨)
# ctgan.fit(X_train, discrete_columns=cat_features)
# 
# # 새로운 데이터 생성
# new_data = ctgan.sample(10)
# new_data

In [19]:
from model_function import cal_params
from model_function import custom_CatBoostClassifier
from model_function import evaluate_

train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
target_value = '복승'
seed = 42

params_list = cal_params(target_value=target_value, seed=seed)

unique_race_ids = train['Race_ID'].unique()
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

models = []
scores = []
for i, (train_race_ids, val_race_ids) in enumerate(kf.split(unique_race_ids)):
    train_ids = unique_race_ids[train_race_ids]
    val_ids = unique_race_ids[val_race_ids]

    train_fold = train[train['Race_ID'].isin(train_ids)].reset_index(drop=True)
    val_fold = train[train['Race_ID'].isin(val_ids)].reset_index(drop=True)

    X_train, y_train, X_val, y_val = all_precoess(train_fold, val_fold, target=target_value, is_train=True)
    
    model = custom_CatBoostClassifier(params_list)
    cat_features = cal_cat_cols(X_train, X_test)
    if i ==0:
        print('범주형 변수: ')
        print(cat_features)
    
    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_features,
    )
    
    models.append(model)
    y_pred = models[i].predict_proba(X_val)
    accuracy = evaluate_(y_pred, y_val, target_value)
    scores.append(accuracy)
    print(f'folds {i+1}')
    print(f'Accuracy: {accuracy}')


print(f'최종 스코어: Avg. Accuracy of validset: {np.mean(scores)}, Std. Accuracy of validset: {np.std(scores)}')

범주형 변수: 
['나이_2번선수', '등급_1번선수', '등급_6번선수', '등급_4번선수', '나이_6번선수', '나이_1번선수', '나이_5번선수', '등급_3번선수', '나이_4번선수', '등급_5번선수', '등급_2번선수', '나이_3번선수']
folds 1
Accuracy: 0.2450281425891182
folds 2
Accuracy: 0.2652908067542214
folds 3
Accuracy: 0.26416510318949343
folds 4
Accuracy: 0.24652908067542215
folds 5
Accuracy: 0.2716697936210131
최종 스코어: Avg. Accuracy of validset: 0.2585365853658536, Std. Accuracy of validset: 0.01073721634010021


In [20]:
def cal_mean_feature_importance(models):
    importance_avg = None
    
    for model in models:
        importance_df = model.print_feature_importance()
        if importance_avg is None:
            importance_avg = importance_df.copy()
        else:
            importance_avg['Importance'] += importance_df['Importance']
    
    importance_avg['Importance'] /= len(models)
    
    importance_avg = importance_avg.sort_values(by='Importance', ascending=False).reset_index(drop=True)
    return importance_avg
    
importance_avg = cal_mean_feature_importance(models)
importance_avg

,Group,Importance
0,모터_평균착순점,1.827470
1,PCA_코스성적_1,1.454722
2,모터_연대율2,1.429754
3,모터_연대율3,1.370844
4,최근6회차_평균착순점,1.278426
5,전일성적,1.075994
6,최근6회차_평균득점,0.893773
7,연간성적_평균착순점,0.825242
8,연간성적_연대율,0.772957
9,보트_평균착순점,0.763427


- Test 데이터

In [21]:
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

In [22]:
unique_race_ids = train['Race_ID'].unique()
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

all_predictions = []
for i, (train_race_ids, val_race_ids) in enumerate(kf.split(unique_race_ids)):
    train_ids = unique_race_ids[train_race_ids]

    train_fold = train[train['Race_ID'].isin(train_ids)].reset_index(drop=True)
    test_fold = test.copy()

    X_train, y_train, X_test, y_test = all_precoess(train_fold, test_fold, target=target_value, is_train=True)
    
    y_pred = models[i].predict_proba(X_test)
    all_predictions.append(y_pred)

contain_연승 = True
mean_pred = np.mean(all_predictions, axis=0)
mean_pred = mean_pred / mean_pred.sum(axis=1, keepdims=True)
accuracy, 연승_score = evaluate_(mean_pred, y_test, target_value, contain_연승=contain_연승)

print(f'최종 스코어: Accuracy of testset: {accuracy}')
if contain_연승:
    print(f'최종 연승 적중률: Accuracy of testset: {연승_score}')

최종 스코어: Accuracy of testset: 0.26745329400196655
최종 연승 적중률: Accuracy of testset: 0.6666666666666666


- Base_line_단승:  Accuracy of testset: 0.4532940019665683
- 

### 복승
- Accuracy of testset: 0.26450344149459193, 연승 적중률: Accuracy of testset: 0.6666666666666666 (base_line optuna)
- Accuracy of testset: 0.2713864306784661, 연승 적중률: Accuracy of testset: 0.6794493608652901 (전일 성적 평균 추가)
- Accuracy of testset: 0.27728613569321536, 연승 적중률: Accuracy of testset: 0.6686332350049164 (코스별 성적 pca + 해당 코스만 남김)
- Accuracy of testset: 0.2753195673549656, 연승 적중률: Accuracy of testset: 0.6794493608652901 (최근 8경기 중 3개는 평균, 나머지 버림)
- Accuracy of testset: 0.2713864306784661, 연승 적중률: Accuracy of testset: 0.6764995083579154 (FL drop)
- Accuracy of testset: 0.2723697148475909, 연승 적중률: Accuracy of testset: 0.6755162241887905 (성별 mapping 남:0, 여:1)
- Accuracy of testset: 0.2743362831858407, 연승 적중률: Accuracy of testset: 0.6774827925270404 (착순 관련 데이터 reverse)
- // Accuracy of testset: 0.2704031465093412, 연승 적중률: Accuracy of testset: 0.671583087512291 (다시 FL 추가) 채택 X
- Accuracy of testset: 0.27630285152409045, 연승 적중률: Accuracy of testset: 0.6755162241887905 (나이 binning)
- Accuracy of testset: 0.28416912487708945, 연승 적중률: Accuracy of testset: 0.6696165191740413 (중량부과여부, 부과된 중량 추가 / 체중은 drop)

- Base_line_삼복승: Accuracy of testset: 0.18682399213372664

# 실제 경기 예측

In [388]:
from crawlling_entry import crawl_race_entries

연도 = 2024
회차 = 36
일차 = 2
sub = crawl_race_entries(연도, 회차, 일차)
for col in sub.columns:
    try:
        sub[col] = sub[col].astype(float)
    except:
        continue

sub

,연도,회차,일차,경주번호,번호,등급,기수,선수명,성별,나이,...,모터번호,모터_평균착순점,모터_연대율2,모터_연대율3,전탑승선수1,전탑승선수2,보트번호,보트_평균착순점,보트_연대율,특이사항
0,2024.0,36.0,2.0,1.0,1.0,B2,17.0,이현준,남,24.0,...,87.0,4.88,20.6,50.0,김도휘/122,손제민/556,44.0,4.68,28.0,
1,2024.0,36.0,2.0,1.0,2.0,B2,17.0,임 건,남,24.0,...,62.0,6.29,46.4,67.9,김민천/2111,권명호/261,99.0,5.42,35.5,
2,2024.0,36.0,2.0,1.0,3.0,B2,17.0,이현지,여,24.0,...,14.0,5.44,38.9,55.6,장영태/2161,이승일/132,108.0,4.60,23.3,
3,2024.0,36.0,2.0,1.0,4.0,B2,17.0,임혜란,여,29.0,...,25.0,5.93,46.4,60.7,이 인/324,이진우/322,70.0,5.29,23.8,F 후 출전
4,2024.0,36.0,2.0,1.0,5.0,B2,17.0,박지윤,여,23.0,...,61.0,5.39,38.7,48.4,박민성/143,김태규/2446,78.0,5.03,41.2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,2024.0,36.0,2.0,17.0,2.0,B1,15.0,한준희,남,30.0,...,88.0,6.32,44.0,68.0,심상철/32,양원준/1643,35.0,3.93,13.8,
98,2024.0,36.0,2.0,17.0,3.0,B2,1.0,길현태,남,48.0,...,93.0,6.37,46.7,63.3,김민준/14,정인교/616,98.0,4.84,32.0,
99,2024.0,36.0,2.0,17.0,4.0,B2,6.0,양원준,남,47.0,...,48.0,5.63,43.8,53.1,최진혁/103,우진수/1632,13.0,5.32,38.2,
100,2024.0,36.0,2.0,17.0,5.0,A1,11.0,박진서,남,43.0,...,26.0,5.97,43.3,60.0,이승일/331,김국흠/243,104.0,3.11,5.6,


In [390]:
train = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))

unique_race_ids = train['Race_ID'].unique()
kf = KFold(n_splits=5, shuffle=True, random_state=seed)

all_predictions = []
for i, (train_race_ids, val_race_ids) in enumerate(kf.split(unique_race_ids)):
    train_ids = unique_race_ids[train_race_ids]

    train_fold = train[train['Race_ID'].isin(train_ids)].reset_index(drop=True)
    test_fold = sub.copy()

    X_train, y_train, X_test = all_precoess(train_fold, test_fold, target=target_value, is_train=False)

    y_pred = models[i].predict_proba(X_test)
    all_predictions.append(y_pred)


mean_pred = np.mean(all_predictions, axis=0)
mean_pred_df = pd.DataFrame(mean_pred)
mean_pred_df.index = [f"{i+1}경기" for i in range(len(mean_pred_df))]
mean_pred_df.columns = [f"{i+1}번" for i in range(mean_pred_df.shape[1])]
mean_pred_df = mean_pred_df.div(mean_pred_df.sum(axis=1), axis=0)
mean_pred_df

,1번,2번,3번,4번,5번,6번
1경기,0.258595,0.244986,0.201082,0.106651,0.097367,0.091319
2경기,0.174406,0.329891,0.170520,0.044509,0.166731,0.113944
3경기,0.306927,0.166287,0.111214,0.119311,0.254322,0.041938
4경기,0.222416,0.220052,0.231239,0.157952,0.053091,0.115250
5경기,0.201291,0.189651,0.217314,0.159482,0.121543,0.110719
6경기,0.286215,0.254961,0.071163,0.239279,0.065589,0.082792
7경기,0.180091,0.210068,0.185714,0.184912,0.147819,0.091397
8경기,0.236154,0.217432,0.148006,0.191480,0.171705,0.035224
9경기,0.212417,0.225224,0.237095,0.217251,0.064956,0.043058
10경기,0.281805,0.242048,0.239164,0.142249,0.063014,0.031719


In [38]:
mean_pred_df.to_excel(f'./result/{연도}_{회차}_{일차}_{target_value}.xlsx', index=True, float_format="%.2f")